# Calculating driving cost using OSRM (Open Source Routing Machine) and get the outpot as a csv file
## --subset the data that will be used in distance matrix calculation--

In [1]:
from access import access, weights, datasets
import logging
import pandas as pd
import geopandas as gpd

import requests
import json

## Get the centroid of MSOA (demand points)

In [2]:
MSOA_points=gpd.read_file('Data/MSOA_Population_Weighted_Centroids_with_population.gpkg?raw=true')
#MSOA_points = MSOA_points.to_crs('epsg:27700')
MSOA_points = MSOA_points.to_crs('epsg:4326')
MSOA_points['MSOA_lon'] = MSOA_points['geometry'].x
MSOA_points['MSOA_lat'] = MSOA_points['geometry'].y

In [3]:
MSOA_points

,objectid,msoa11cd,pop0-49,pop50-59,pop60-69,pop70-79,pop80-89,pop90+,total_pop,geometry,MSOA_lon,MSOA_lat
0,1,E02002536,5688,1416,1132,891,540,104,9771,POINT (-1.29577 54.61069),-1.295773,54.610691
1,2,E02002537,5578,1251,836,545,386,45,8641,POINT (-1.27726 54.61131),-1.277263,54.611313
2,3,E02002534,2547,824,863,967,407,85,5693,POINT (-1.05346 54.52765),-1.053458,54.527646
3,4,E02002535,4959,1636,1385,870,416,70,9336,POINT (-1.28729 54.62215),-1.287295,54.622148
4,5,E02002532,3783,970,810,655,342,95,6655,POINT (-1.05793 54.53718),-1.057931,54.537176
...,...,...,...,...,...,...,...,...,...,...,...,...
6786,6787,E02004669,3697,1171,1156,946,433,93,7496,POINT (-2.09642 51.94596),-2.096424,51.945960
6787,6788,E02006096,3893,1086,805,804,344,69,7001,POINT (-2.95612 50.87855),-2.956122,50.878552
6788,6789,E02003088,4007,1257,1230,1194,577,179,8444,POINT (-2.81548 51.32678),-2.815479,51.326784
6789,6790,E02006070,5161,1163,906,670,316,75,8291,POINT (-2.97724 51.12711),-2.977239,51.127113


In [4]:
MSOA_points_df = MSOA_points[['msoa11cd','pop60-69','MSOA_lon','MSOA_lat']]
MSOA_points_df

,msoa11cd,pop60-69,MSOA_lon,MSOA_lat
0,E02002536,1132,-1.295773,54.610691
1,E02002537,836,-1.277263,54.611313
2,E02002534,863,-1.053458,54.527646
3,E02002535,1385,-1.287295,54.622148
4,E02002532,810,-1.057931,54.537176
...,...,...,...,...
6786,E02004669,1156,-2.096424,51.945960
6787,E02006096,805,-2.956122,50.878552
6788,E02003088,1230,-2.815479,51.326784
6789,E02006070,906,-2.977239,51.127113


## read vaccination site point data (supply points)

In [5]:
site = gpd.read_file('https://github.com/LingruFeng/dissertation/blob/main/data_github/site_2_26_clean/vaccination_site.gpkg?raw=true')
# site = site.to_crs('epsg:27700')
site = site.to_crs('epsg:4326')
site['site_lon'] = site['geometry'].x
site['site_lat'] = site['geometry'].y

In [6]:
site_df = site[['index','site_lon','site_lat']]
site_df

,index,site_lon,site_lat
0,0,-0.471211,52.128622
1,1,-0.474637,51.894793
2,2,-0.093771,51.468063
3,3,-0.093771,51.468063
4,4,-0.484367,51.891807
...,...,...,...
1595,1595,-4.865818,50.515949
1596,1596,-0.205835,51.433441
1597,1597,-0.173830,51.514861
1598,1598,-1.879170,50.716914


In [7]:
site

,index,supply_value,geometry,site_lon,site_lat
0,0,1,POINT (-0.47121 52.12862),-0.471211,52.128622
1,1,1,POINT (-0.47464 51.89479),-0.474637,51.894793
2,2,1,POINT (-0.09377 51.46806),-0.093771,51.468063
3,3,1,POINT (-0.09377 51.46806),-0.093771,51.468063
4,4,1,POINT (-0.48437 51.89181),-0.484367,51.891807
...,...,...,...,...,...
1595,1595,1,POINT (-4.86582 50.51595),-4.865818,50.515949
1596,1596,1,POINT (-0.20584 51.43344),-0.205835,51.433441
1597,1597,1,POINT (-0.17383 51.51486),-0.173830,51.514861
1598,1598,1,POINT (-1.87917 50.71691),-1.879170,50.716914


## Calculate the Euclidean distance from demand to supply locations 
## --threshold: 30 miles (48280.32 meters)--

In [8]:
# Load each of the example datasets which correspond to the demand (population) and supply ('hospital','pcn','pharmacy','vc') respectively.
distance = access(demand_df = MSOA_points,
           demand_index='msoa11cd',
           demand_value='pop60-69',
           supply_df= site,
           supply_index= 'index',
           supply_value=['supply_value']
                 )
# reproject
distance.demand_df = distance.demand_df.to_crs('epsg:27700')
distance.supply_df = distance.supply_df.to_crs('epsg:27700')

In [9]:
##### Take 48280.32 meters (30 miles) as the calculation threshold
### create_euclidean_distance is a function from the 'access' module
distance.create_euclidean_distance(threshold = 48280.32)
distance.cost_df

,euclidean,origin,dest
0,4573.215379,E02002536,122
1,47458.884713,E02002536,123
2,40875.300003,E02002536,126
3,8449.406849,E02002536,127
4,41028.588815,E02002536,128
...,...,...,...
1674418,33156.644969,E02006679,1458
1674419,38028.236846,E02006679,1460
1674420,19760.429760,E02006679,1591
1674421,40271.511017,E02006679,1592


In [10]:
euclidean_distance_matrix = distance.cost_df

In [11]:
euclidean_distance_matrix = pd.merge(left = euclidean_distance_matrix, 
                                     right = site_df, 
                                     how = 'left', 
                                     left_on = 'dest', 
                                     right_on = 'index')

euclidean_distance_matrix = pd.merge(left = euclidean_distance_matrix, 
                                     right = MSOA_points_df, 
                                     how = 'left', 
                                     left_on = 'origin', 
                                     right_on = 'msoa11cd')

In [12]:
euclidean_distance_matrix.drop(columns=['msoa11cd','index','pop60-69'], axis=1,inplace=True)
euclidean_distance_matrix

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
0,4573.215379,E02002536,122,-1.347271,54.582497,-1.295773,54.610691
1,47458.884713,E02002536,123,-1.467913,55.025422,-1.295773,54.610691
2,40875.300003,E02002536,126,-1.582189,54.938577,-1.295773,54.610691
3,8449.406849,E02002536,127,-1.214583,54.551181,-1.295773,54.610691
4,41028.588815,E02002536,128,-1.427770,54.971409,-1.295773,54.610691
...,...,...,...,...,...,...,...
1674418,33156.644969,E02006679,1458,-1.761545,51.548116,-2.136956,51.364067
1674419,38028.236846,E02006679,1460,-1.619865,51.255033,-2.136956,51.364067
1674420,19760.429760,E02006679,1591,-2.407209,51.418670,-2.136956,51.364067
1674421,40271.511017,E02006679,1592,-1.800300,51.070083,-2.136956,51.364067


In [25]:
euclidean_distance_matrix.to_csv('euclidean_distance_30mile.csv',index=False)

In [13]:
euclidean_distance_matrix.shape

(1674423, 7)

**The following cells are not used.**

In [62]:
euclidean_distance_matrix_sub=euclidean_distance_matrix[0:5]
euclidean_distance_matrix_sub

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
0,4573.215379,E02002536,122,-1.347271,54.582497,-1.295773,54.610691
1,47458.884713,E02002536,123,-1.467913,55.025422,-1.295773,54.610691
2,40875.300003,E02002536,126,-1.582189,54.938577,-1.295773,54.610691
3,8449.406849,E02002536,127,-1.214583,54.551181,-1.295773,54.610691
4,41028.588815,E02002536,128,-1.427770,54.971409,-1.295773,54.610691


In [79]:
# An example for 5 requests
distance_list = []
origin_id_list = []
destination_id_list = []

for (i1, row1) in euclidean_distance_matrix_sub.iterrows():
    LatOrigin = str(row1['MSOA_lat'])
    LongOrigin = str(row1['MSOA_lon'])
    origin_id = row1['origin']
    
    LatDestination = str(row1['site_lat'])
    LongDestination = str(row1['site_lon'])
    destination_id = row1['dest']
    
    url = "http://82.11.51.26:5000/route/v1/driving/" + LongOrigin +","+ LatOrigin + ";" + LongDestination +","+ LatDestination + "?steps=false"
    result_distance = requests.get(url).json()['routes'][0]['distance']
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

output = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output['origin_id'] = origin_id_list
output['destination_id'] = destination_id_list
output

,Distance in meter,origin_id,destination_id
0,6712.7,E02002536,122
1,56619.8,E02002536,123
2,51152.7,E02002536,126
3,13881.2,E02002536,127
4,50625.1,E02002536,128


# An API request 100,000 times, divided into 17 parts to request all the distance (total 1,674,423)

## sub 0 — — 100000 :

In [14]:
euclidean_distance_matrix_sub01=euclidean_distance_matrix[0:2000]
euclidean_distance_matrix_sub02=euclidean_distance_matrix[2000:10000]
euclidean_distance_matrix_sub03=euclidean_distance_matrix[10000:40000]
euclidean_distance_matrix_sub04=euclidean_distance_matrix[40000:100000]

In [15]:
euclidean_distance_matrix_sub01

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
0,4573.215379,E02002536,122,-1.347271,54.582497,-1.295773,54.610691
1,47458.884713,E02002536,123,-1.467913,55.025422,-1.295773,54.610691
2,40875.300003,E02002536,126,-1.582189,54.938577,-1.295773,54.610691
3,8449.406849,E02002536,127,-1.214583,54.551181,-1.295773,54.610691
4,41028.588815,E02002536,128,-1.427770,54.971409,-1.295773,54.610691
...,...,...,...,...,...,...,...
1995,45171.591572,E02002516,762,-1.576421,54.891925,-1.073841,54.608615
1996,39246.295669,E02002516,835,-1.400237,54.906523,-1.073841,54.608615
1997,12507.984356,E02002516,836,-1.213812,54.686367,-1.073841,54.608615
1998,10092.156164,E02002516,838,-1.216531,54.571757,-1.073841,54.608615


### 0-2000：

In [16]:
distance_list = []
origin_id_list = []
destination_id_list = []

for (i1, row1) in euclidean_distance_matrix_sub01.iterrows():
    LatOrigin = str(row1['MSOA_lat'])
    LongOrigin = str(row1['MSOA_lon'])
    origin_id = row1['origin']
    
    LatDestination = str(row1['site_lat'])
    LongDestination = str(row1['site_lon'])
    destination_id = row1['dest']
    
    url = "http://82.11.51.26:5000/route/v1/driving/" + LongOrigin +","+ LatOrigin + ";" + LongDestination +","+ LatDestination + "?steps=false"
    result_distance = requests.get(url).json()['routes'][0]['distance']
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

output01 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output01['origin_id'] = origin_id_list
output01['destination_id'] = destination_id_list
output01

,Distance in meter,origin_id,destination_id
0,6712.7,E02002536,122
1,56619.8,E02002536,123
2,51152.7,E02002536,126
3,13881.2,E02002536,127
4,50625.1,E02002536,128
...,...,...,...
1995,69599.4,E02002516,762
1996,60828.5,E02002516,835
1997,34861.7,E02002516,836
1998,12710.9,E02002516,838


In [32]:
if len(output01[output01['Distance in meter'].isnull()==True])==0:
    print('There is no empty result.')

There is no empty result.


### 2000-10000：

In [17]:
distance_list = []
origin_id_list = []
destination_id_list = []

for (i1, row1) in euclidean_distance_matrix_sub02.iterrows():
    LatOrigin = str(row1['MSOA_lat'])
    LongOrigin = str(row1['MSOA_lon'])
    origin_id = row1['origin']
    
    LatDestination = str(row1['site_lat'])
    LongDestination = str(row1['site_lon'])
    destination_id = row1['dest']
    
    url = "http://82.11.51.26:5000/route/v1/driving/" + LongOrigin +","+ LatOrigin + ";" + LongDestination +","+ LatDestination + "?steps=false"
    result_distance = requests.get(url).json()['routes'][0]['distance']
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

output02 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output02['origin_id'] = origin_id_list
output02['destination_id'] = destination_id_list
output02

,Distance in meter,origin_id,destination_id
0,21173.0,E02002516,843
1,13421.5,E02002516,844
2,38513.7,E02002516,845
3,21173.0,E02002516,846
4,13421.5,E02002516,849
...,...,...,...
7995,48411.2,E02005241,187
7996,16785.8,E02005241,189
7997,16785.8,E02005241,190
7998,51534.5,E02005241,192


In [31]:
if len(output02[output02['Distance in meter'].isnull()==True])==0:
    print('There is no empty result.')

There is no empty result.


### 10000-40000:

In [33]:
distance_list = []
origin_id_list = []
destination_id_list = []

for (i1, row1) in euclidean_distance_matrix_sub03.iterrows():
    LatOrigin = str(row1['MSOA_lat'])
    LongOrigin = str(row1['MSOA_lon'])
    origin_id = row1['origin']
    
    LatDestination = str(row1['site_lat'])
    LongDestination = str(row1['site_lon'])
    destination_id = row1['dest']
    
    url = "http://82.11.51.26:5000/route/v1/driving/" + LongOrigin +","+ LatOrigin + ";" + LongDestination +","+ LatDestination + "?steps=false"
    result_distance = requests.get(url).json()['routes'][0]['distance']
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

output03 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output03['origin_id'] = origin_id_list
output03['destination_id'] = destination_id_list

if len(output03[output03['Distance in meter'].isnull()==True])==0:
    print('There is no empty result.')
output03

There is no empty result.


,Distance in meter,origin_id,destination_id
0,54310.8,E02005241,201
1,60490.6,E02005241,620
2,14400.3,E02005241,698
3,44572.7,E02005241,699
4,37814.6,E02005241,700
...,...,...,...
29995,48497.6,E02003879,941
29996,40698.7,E02003879,943
29997,38838.4,E02003879,944
29998,33918.8,E02003879,945


### 40000-100000:

In [34]:
distance_list = []
origin_id_list = []
destination_id_list = []

for (i1, row1) in euclidean_distance_matrix_sub04.iterrows():
    LatOrigin = str(row1['MSOA_lat'])
    LongOrigin = str(row1['MSOA_lon'])
    origin_id = row1['origin']
    
    LatDestination = str(row1['site_lat'])
    LongDestination = str(row1['site_lon'])
    destination_id = row1['dest']
    
    url = "http://82.11.51.26:5000/route/v1/driving/" + LongOrigin +","+ LatOrigin + ";" + LongDestination +","+ LatDestination + "?steps=false"
    result_distance = requests.get(url).json()['routes'][0]['distance']
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

output04 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output04['origin_id'] = origin_id_list
output04['destination_id'] = destination_id_list

if len(output04[output04['Distance in meter'].isnull()==True])==0:
    print('There is no empty result.')
output04

There is no empty result.


,Distance in meter,origin_id,destination_id
0,57117.5,E02003879,952
1,21646.3,E02003879,953
2,26272.4,E02003879,955
3,37099.7,E02003879,957
4,30190.9,E02003879,959
...,...,...,...
59995,37304.2,E02001382,170
59996,12059.4,E02001382,171
59997,25517.6,E02001382,172
59998,18000.5,E02001382,173


### Concat 

In [35]:
output0_100000 = pd.concat([output01, output02, output03, output04], axis=0, ignore_index=True)
output0_100000

,Distance in meter,origin_id,destination_id
0,6712.7,E02002536,122
1,56619.8,E02002536,123
2,51152.7,E02002536,126
3,13881.2,E02002536,127
4,50625.1,E02002536,128
...,...,...,...
99995,37304.2,E02001382,170
99996,12059.4,E02001382,171
99997,25517.6,E02001382,172
99998,18000.5,E02001382,173


### Export

In [36]:
output0_100000.to_csv('Distance0_100000.csv',index=False)

## Sub 100000-200000:

In [38]:
euclidean_distance_matrix_sub11=euclidean_distance_matrix[100000:150000]
euclidean_distance_matrix_sub12=euclidean_distance_matrix[150000:200000]
print(euclidean_distance_matrix_sub11.shape,euclidean_distance_matrix_sub12.shape)

(50000, 7) (50000, 7)


In [39]:
euclidean_distance_matrix_sub11

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
100000,44245.470105,E02001382,176,-2.227739,53.460757,-2.885853,53.400095
100001,24474.870365,E02001382,179,-2.629406,53.558178,-2.885853,53.400095
100002,43838.218639,E02001382,183,-2.228165,53.429565,-2.885853,53.400095
100003,45509.882270,E02001382,184,-2.229486,53.517882,-2.885853,53.400095
100004,45509.882270,E02001382,185,-2.229486,53.517882,-2.885853,53.400095
...,...,...,...,...,...,...,...
149995,30060.610297,E02002471,703,-1.751687,53.750311,-1.333007,53.644175
149996,35633.733275,E02002471,705,-1.766658,53.835174,-1.333007,53.644175
149997,33508.554343,E02002471,707,-1.835138,53.686297,-1.333007,53.644175
149998,34674.528682,E02002471,710,-1.831082,53.742890,-1.333007,53.644175


In [40]:
euclidean_distance_matrix_sub12

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
150000,25313.932868,E02002471,730,-0.954249,53.611407,-1.333007,53.644175
150001,24614.098341,E02002471,738,-1.700052,53.607625,-1.333007,53.644175
150002,30716.036242,E02002471,739,-1.784087,53.578854,-1.333007,53.644175
150003,23613.536559,E02002471,745,-1.547287,53.814169,-1.333007,53.644175
150004,14435.993309,E02002471,749,-1.370126,53.772041,-1.333007,53.644175
...,...,...,...,...,...,...,...
199995,32928.868095,E02006512,650,-1.908642,52.472649,-1.716218,52.201091
199996,40826.255207,E02006512,653,-1.882875,52.553667,-1.716218,52.201091
199997,27761.077571,E02006512,655,-1.505210,52.414538,-1.716218,52.201091
199998,28036.442770,E02006512,656,-1.474920,52.405173,-1.716218,52.201091


### 100000-150000

In [41]:
distance_list = []
origin_id_list = []
destination_id_list = []

for (i1, row1) in euclidean_distance_matrix_sub11.iterrows():
    LatOrigin = str(row1['MSOA_lat'])
    LongOrigin = str(row1['MSOA_lon'])
    origin_id = row1['origin']
    
    LatDestination = str(row1['site_lat'])
    LongDestination = str(row1['site_lon'])
    destination_id = row1['dest']
    
    url = "http://82.11.51.26:5000/route/v1/driving/" + LongOrigin +","+ LatOrigin + ";" + LongDestination +","+ LatDestination + "?steps=false"
    result_distance = requests.get(url).json()['routes'][0]['distance']
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

output11 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output11['origin_id'] = origin_id_list
output11['destination_id'] = destination_id_list

if len(output11[output11['Distance in meter'].isnull()==True])==0:
    print('There is no empty result.')
output11

There is no empty result.


,Distance in meter,origin_id,destination_id
0,51725.7,E02001382,176
1,42254.3,E02001382,179
2,58028.3,E02001382,183
3,53652.8,E02001382,184
4,53652.8,E02001382,185
...,...,...,...
49995,38513.5,E02002471,703
49996,44247.4,E02002471,705
49997,49199.9,E02002471,707
49998,44177.2,E02002471,710


### 150000-200000

In [42]:
distance_list = []
origin_id_list = []
destination_id_list = []

for (i1, row1) in euclidean_distance_matrix_sub12.iterrows():
    LatOrigin = str(row1['MSOA_lat'])
    LongOrigin = str(row1['MSOA_lon'])
    origin_id = row1['origin']
    
    LatDestination = str(row1['site_lat'])
    LongDestination = str(row1['site_lon'])
    destination_id = row1['dest']
    
    url = "http://82.11.51.26:5000/route/v1/driving/" + LongOrigin +","+ LatOrigin + ";" + LongDestination +","+ LatDestination + "?steps=false"
    result_distance = requests.get(url).json()['routes'][0]['distance']
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

output12 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output12['origin_id'] = origin_id_list
output12['destination_id'] = destination_id_list

if len(output12[output12['Distance in meter'].isnull()==True])==0:
    print('There is no empty result.')
output12

There is no empty result.


,Distance in meter,origin_id,destination_id
0,40403.8,E02002471,730
1,32970.8,E02002471,738
2,37054.8,E02002471,739
3,29884.8,E02002471,745
4,16817.1,E02002471,749
...,...,...,...
49995,49120.0,E02006512,650
49996,61958.1,E02006512,653
49997,31883.6,E02006512,655
49998,32370.3,E02006512,656


### Concat

In [43]:
output100000_200000 = pd.concat([output11, output12], axis=0, ignore_index=True)
output100000_200000

,Distance in meter,origin_id,destination_id
0,51725.7,E02001382,176
1,42254.3,E02001382,179
2,58028.3,E02001382,183
3,53652.8,E02001382,184
4,53652.8,E02001382,185
...,...,...,...
99995,49120.0,E02006512,650
99996,61958.1,E02006512,653
99997,31883.6,E02006512,655
99998,32370.3,E02006512,656


### Export

In [44]:
output100000_200000.to_csv('Distance100000_200000.csv',index=False)

## Sub 200000-300000:

In [45]:
euclidean_distance_matrix_sub21=euclidean_distance_matrix[200000:300000]
print(euclidean_distance_matrix_sub21.shape)

(100000, 7)


In [46]:
euclidean_distance_matrix_sub21

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
200000,36891.923095,E02006512,775,-1.251387,52.370617,-1.716218,52.201091
200001,31014.839486,E02006512,1079,-1.535103,51.945561,-1.716218,52.201091
200002,30850.828892,E02006512,1090,-1.321326,52.067385,-1.716218,52.201091
200003,41977.444480,E02006512,1250,-2.068761,51.892574,-1.716218,52.201091
200004,47159.696403,E02006512,1252,-2.171492,51.883356,-1.716218,52.201091
...,...,...,...,...,...,...,...
299995,34573.962619,E02004868,1413,-0.142644,51.511259,-0.446670,51.757856
299996,32845.753954,E02004868,1415,-0.196671,51.506891,-0.446670,51.757856
299997,42913.763429,E02004868,1416,0.055615,51.531648,-0.446670,51.757856
299998,34492.484551,E02004868,1417,-0.183086,51.494729,-0.446670,51.757856


In [47]:
distance_list = []
origin_id_list = []
destination_id_list = []

for (i1, row1) in euclidean_distance_matrix_sub21.iterrows():
    LatOrigin = str(row1['MSOA_lat'])
    LongOrigin = str(row1['MSOA_lon'])
    origin_id = row1['origin']
    
    LatDestination = str(row1['site_lat'])
    LongDestination = str(row1['site_lon'])
    destination_id = row1['dest']
    
    url = "http://82.11.51.26:5000/route/v1/driving/" + LongOrigin +","+ LatOrigin + ";" + LongDestination +","+ LatDestination + "?steps=false"
    result_distance = requests.get(url).json()['routes'][0]['distance']
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

output21 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output21['origin_id'] = origin_id_list
output21['destination_id'] = destination_id_list

if len(output21[output21['Distance in meter'].isnull()==True])==0:
    print('There is no empty result.')
output21

There is no empty result.


,Distance in meter,origin_id,destination_id
0,47414.3,E02006512,775
1,36544.7,E02006512,1079
2,43340.3,E02006512,1090
3,55322.9,E02006512,1250
4,61884.0,E02006512,1252
...,...,...,...
99995,41573.1,E02004868,1413
99996,39861.6,E02004868,1415
99997,65684.0,E02004868,1416
99998,44369.5,E02004868,1417


### Export

In [48]:
output21.to_csv('Distance200000_300000.csv',index=False)

## Sub 300000-400000:

In [49]:
euclidean_distance_matrix_sub3=euclidean_distance_matrix[300000:400000]
print(euclidean_distance_matrix_sub3.shape)

(100000, 7)


### 300000-400000

In [50]:
distance_list = []
origin_id_list = []
destination_id_list = []

for (i1, row1) in euclidean_distance_matrix_sub3.iterrows():
    LatOrigin = str(row1['MSOA_lat'])
    LongOrigin = str(row1['MSOA_lon'])
    origin_id = row1['origin']
    
    LatDestination = str(row1['site_lat'])
    LongDestination = str(row1['site_lon'])
    destination_id = row1['dest']
    
    url = "http://82.11.51.26:5000/route/v1/driving/" + LongOrigin +","+ LatOrigin + ";" + LongDestination +","+ LatDestination + "?steps=false"
    result_distance = requests.get(url).json()['routes'][0]['distance']
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

output3 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output3['origin_id'] = origin_id_list
output3['destination_id'] = destination_id_list

if len(output3[output3['Distance in meter'].isnull()==True])==0:
    print('There is no empty result.')
output3

There is no empty result.


,Distance in meter,origin_id,destination_id
0,40267.5,E02004868,1419
1,58202.8,E02004868,1423
2,36584.9,E02004868,1426
3,66988.7,E02004868,1427
4,27400.8,E02004868,1428
...,...,...,...
99995,48106.7,E02004476,1305
99996,42676.7,E02004476,1306
99997,46471.9,E02004476,1307
99998,63372.2,E02004476,1327


### Export

In [51]:
output3.to_csv('Distance300000_400000.csv',index=False)

## Sub 400000-1000000:

In [52]:
euclidean_distance_matrix_sub4=euclidean_distance_matrix[400000:1000000]
print(euclidean_distance_matrix_sub4.shape)

(600000, 7)


In [53]:
euclidean_distance_matrix_sub4

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
400000,46961.607443,E02004476,1330,-0.096077,51.630967,0.572189,51.560565
400001,35815.803666,E02004476,1334,0.068553,51.490265,0.572189,51.560565
400002,43147.659577,E02004476,1335,-0.012818,51.429889,0.572189,51.560565
400003,45936.987126,E02004476,1341,-0.090231,51.557610,0.572189,51.560565
400004,39991.080969,E02004476,1346,0.000222,51.607964,0.572189,51.560565
...,...,...,...,...,...,...,...
999995,13448.416675,E02000627,1508,-0.297689,51.519983,-0.122486,51.468451
999996,4460.395058,E02000627,1510,-0.088010,51.502280,-0.122486,51.468451
999997,4342.724508,E02000627,1511,-0.127758,51.507351,-0.122486,51.468451
999998,12690.004055,E02000627,1513,-0.261888,51.542246,-0.122486,51.468451


In [54]:
distance_list = []
origin_id_list = []
destination_id_list = []

for (i1, row1) in euclidean_distance_matrix_sub4.iterrows():
    LatOrigin = str(row1['MSOA_lat'])
    LongOrigin = str(row1['MSOA_lon'])
    origin_id = row1['origin']
    
    LatDestination = str(row1['site_lat'])
    LongDestination = str(row1['site_lon'])
    destination_id = row1['dest']
    
    url = "http://82.11.51.26:5000/route/v1/driving/" + LongOrigin +","+ LatOrigin + ";" + LongDestination +","+ LatDestination + "?steps=false"
    result_distance = requests.get(url).json()['routes'][0]['distance']
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

output4 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output4['origin_id'] = origin_id_list
output4['destination_id'] = destination_id_list

if len(output4[output4['Distance in meter'].isnull()==True])==0:
    print('There is no empty result.')
output4

There is no empty result.


,Distance in meter,origin_id,destination_id
0,60495.7,E02004476,1330
1,47252.4,E02004476,1334
2,53887.3,E02004476,1335
3,56202.6,E02004476,1341
4,55395.4,E02004476,1346
...,...,...,...
599995,18845.8,E02000627,1508
599996,5098.7,E02000627,1510
599997,5777.0,E02000627,1511
599998,16527.6,E02000627,1513


### Export

In [55]:
output4.to_csv('Distance400000_1000000.csv',index=False)

## Sub 1000000-1674423:

In [59]:
euclidean_distance_matrix_sub5=euclidean_distance_matrix[1000000:1674423]
print(euclidean_distance_matrix_sub5.shape)

(674423, 7)


In [60]:
euclidean_distance_matrix_sub5

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
1000000,5263.435352,E02000627,1516,-0.125743,51.515723,-0.122486,51.468451
1000001,12202.384569,E02000627,1517,0.053108,51.470757,-0.122486,51.468451
1000002,5933.484711,E02000627,1518,-0.206091,51.457594,-0.122486,51.468451
1000003,6423.683632,E02000627,1519,-0.093095,51.523205,-0.122486,51.468451
1000004,11477.242091,E02000627,1521,0.029846,51.508460,-0.122486,51.468451
...,...,...,...,...,...,...,...
1674418,33156.644969,E02006679,1458,-1.761545,51.548116,-2.136956,51.364067
1674419,38028.236846,E02006679,1460,-1.619865,51.255033,-2.136956,51.364067
1674420,19760.429760,E02006679,1591,-2.407209,51.418670,-2.136956,51.364067
1674421,40271.511017,E02006679,1592,-1.800300,51.070083,-2.136956,51.364067


In [61]:
distance_list = []
origin_id_list = []
destination_id_list = []

for (i1, row1) in euclidean_distance_matrix_sub5.iterrows():
    LatOrigin = str(row1['MSOA_lat'])
    LongOrigin = str(row1['MSOA_lon'])
    origin_id = row1['origin']
    
    LatDestination = str(row1['site_lat'])
    LongDestination = str(row1['site_lon'])
    destination_id = row1['dest']
    
    url = "http://82.11.51.26:5000/route/v1/driving/" + LongOrigin +","+ LatOrigin + ";" + LongDestination +","+ LatDestination + "?steps=false"
    result_distance = requests.get(url).json()['routes'][0]['distance']
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

output5 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output5['origin_id'] = origin_id_list
output5['destination_id'] = destination_id_list

if len(output5[output5['Distance in meter'].isnull()==True])==0:
    print('There is no empty result.')
output5

There is no empty result.


,Distance in meter,origin_id,destination_id
0,6525.7,E02000627,1516
1,13144.1,E02000627,1517
2,6683.7,E02000627,1518
3,7649.8,E02000627,1519
4,14939.2,E02000627,1521
...,...,...,...
674418,48512.4,E02006679,1458
674419,43150.5,E02006679,1460
674420,26966.3,E02006679,1591
674421,49329.1,E02006679,1592


### Export

In [62]:
output5.to_csv('Distance1000000_1674423.csv',index=False)

# Travel Cost all together and then export to a csv file

In [2]:
# These are data generated by Google_distance_cost.ipynb
distance_matrix0 = pd.read_csv('Distance0_100000.csv')
distance_matrix1 = pd.read_csv('Distance100000_200000.csv')
distance_matrix2 = pd.read_csv('Distance200000_300000.csv')
distance_matrix3 = pd.read_csv('Distance300000_400000.csv')
distance_matrix4 = pd.read_csv('Distance400000_1000000.csv')
distance_matrix5 = pd.read_csv('Distance1000000_1674423.csv')

In [3]:
distance_matrix = pd.concat([distance_matrix0, distance_matrix1,distance_matrix2,distance_matrix3,distance_matrix4,distance_matrix5], axis=0, ignore_index=True)
distance_matrix

,Distance in meter,origin_id,destination_id
0,6712.7,E02002536,122
1,56619.8,E02002536,123
2,51152.7,E02002536,126
3,13881.2,E02002536,127
4,50625.1,E02002536,128
...,...,...,...
1674418,48512.4,E02006679,1458
1674419,43150.5,E02006679,1460
1674420,26966.3,E02006679,1591
1674421,49329.1,E02006679,1592


In [4]:
# check the data
distance_matrix.isna().sum()

Distance in meter    0
origin_id            0
destination_id       0
dtype: int64

In [5]:
# check the data
distance_matrix[distance_matrix['Distance in meter']==0]

,Distance in meter,origin_id,destination_id
117366,0.0,E02006875,1320


In [6]:
distance_matrix.to_csv('distance_cost_final_OSRM.csv',index=False)